In [6]:
from src.cloud_functions._1_fetch_data._1_13_fetch_items_promotions.main import fetch_promotions_data


In [7]:
class MockRequest:
    
    def __init__(self, json_data):
        self._json_data = json_data

    def get_json(self):
        return self._json_data

In [8]:
test_data = {
 "access_token": None,
 "client_id": "4959083987776428",
 "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
 "seller_id": 189643563,
 "store_name": "hubsmarthome"
}

mock_request = MockRequest(test_data)

In [9]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime
import time

semaphore = asyncio.Semaphore(100)  # Control the number of simultaneous requests

async def main_async(request):
    # Parsing request data
    data = request.get_json()
    client_id = data.get('client_id')
    client_secret = data.get('client_secret')
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')
    access_token = data.get('access_token')
    print('** Defining authentication... **')
    
    # Authenticate (assuming this is now centralized in utils.py or a similar file)
    if not access_token:
        access_token = authenticate(client_id, client_secret)  # You can add this to a common module
    print('** Connecting to storage and BigQuery... **')

    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    #bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    
    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_ITEM_PROMOTION
    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')
    
    # Fetch item IDs from the storage bucket
    blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
    items_id = fetch_items_from_storage(storage, settings.BUCKET_STORES, blob_items_prefix, 'id')
    print(f'** Items found: {len(items_id)}**')
    
    print(f'** Cleaning blob **')
    # Path for saving promotions details
    # marketplace
    blob_basic_path_marketplace = settings.BLOB_PROMOTIONS(store_name)
    date_blob_path_marketplace = f'{blob_basic_path_marketplace}date={today_str}/'
    
    # mshops
    blob_basic_path_mshops = settings.BLOB_PROMOTIONS_MSHOPS(store_name)
    date_blob_path_mshops = f'{blob_basic_path_mshops}date={today_str}/'
    
    # Clean existing files in the storage bucket
    storage.clean_blobs(bucket_name, date_blob_path_marketplace)
    storage.clean_blobs(bucket_name, date_blob_path_mshops)
    print(f'** Starting API requests for {len(items_id)} items**')
    # URL function for API
    url_marketplace = settings.URL_PROMOTIONS_MARKETPLACE
    url_mshops = settings.URL_PROMOTIONS_MSHOPS
    headers = {'Authorization': f'Bearer {access_token}'}
    
    # Batch processing the API requests
    # PROMOTIONS MELI
    try:
        async with aiohttp.ClientSession() as session:
            await batch_process(session, items_id, url_marketplace, headers, bucket_name, date_blob_path_marketplace, storage,  sleep_time=15)
    except aiohttp.ClientResponseError as e:
        if e.status == 429:
            retry_after = e.headers.get('Retry-After')
            if retry_after:
                wait_time = int(retry_after)
                print(f"Limite de taxa atingido. Aguardando {wait_time} segundos.")
                time.sleep(wait_time)
                # Retry after timeout
                await batch_process(session, items_id, url_marketplace, headers, bucket_name, date_blob_path_marketplace, storage, add_item_id = True)
            else:
                print("Limite de taxa atingido, mas nenhum cabeçalho Retry-After foi fornecido. Aguardando um tempo padrão.")
                time.sleep(10)  # Wait for default time if header is not present

    #time.sleep(60)
    
    # Batch processing the API requests
    # PROMOTIONS SHOPS
    try:

        async with aiohttp.ClientSession() as session:
            await batch_process(session, items_id, url_mshops, headers, bucket_name, date_blob_path_mshops, storage,  sleep_time=15)
    except aiohttp.ClientResponseError as e:
        if e.status == 429:
            retry_after = e.headers.get('Retry-After')
            if retry_after:
                wait_time = int(retry_after)
                print(f"Limite de taxa atingido. Aguardando {wait_time} segundos.")
                time.sleep(wait_time)
                # Retry after timeout
                await batch_process(session, items_id, url_marketplace, headers, bucket_name, date_blob_path_marketplace, storage, add_item_id = True)
            else:
                print("Limite de taxa atingido, mas nenhum cabeçalho Retry-After foi fornecido. Aguardando um tempo padrão.")
                time.sleep(10)  # Wait for default time if header is not present

    
    print('** Logging process in management table... **')
    # Log the process in BigQuery
    log_process(seller_id, destiny_table, today_str, table_management, processed_to_bq=False)

    return ('Success', 200)

def fetch_promotions_data(request):
    return (main_async(request))



In [10]:
result = await fetch_promotions_data(mock_request)


** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: D:/bacar/Savvi/GLM/Desenvolvimento/service_account/service_account_datalakev2.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-25/batch_0__process_time=2024-10-25T07:00:00.267180-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-25/batch_10__process_time=2024-10-25T07:00:00.282616-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-25/batch_1__process_time=2024-10-25T07:00:00.745113-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-25/batch_2__process_time=2024-10-25T07:00:00.327130-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-25/batch_3__process_time=2024-10-25T07:00:00.669467-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-25/batch_4__process_time=2024-10-25T07:00:00.941750-03:00.json
Reading fil

TypeError: Object of type ClientConnectorError is not JSON serializable

In [63]:
print(result)


('Success', 200)


In [2]:
import os
from src.config.settings import *

path_credentials = PATH_SERVICE_ACCOUNT

if os.path.exists(path_credentials):
    print("O arquivo existe.")
else:
    print("O arquivo não foi encontrado.")


O arquivo existe.
